***
# MAKE PREDICTIONS USING THE FASTF1 API
***

#### _Disclaimer_: We'll make extensive use of custom made Python functions (residing in the fastf1_helper.py file). Those functions will do the tedious work of loading the data from each race, aggregating columns (average tap times etc), and they're not strictly necessary to understand the ML workflow.

- #### Import the necessary libraries (all imports will be done here)

In [1]:
import fastf1
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import xgboost as xgb
from fastf1_helper import get_race, get_season

- #### Create a path for cached data, so that it doesn't constantly re-download the same files

In [2]:
my_path = r'C:\Users\apost\miniconda3\envs\fastF1_cache'
fastf1.Cache.enable_cache(my_path)

- #### Load the seasons of 2023, 2024, 2024

In [3]:
stats_2023 = get_season(2023)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data

In [4]:
stats_2024 = get_season(2024)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data 

In [5]:
stats_2025 = get_season(2025)

core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22

In [6]:
stats_2023.isna().sum(), stats_2024.isna().sum(), stats_2025.isna().sum()

(Driver            0
 lapsCompleted     0
 Team              0
 CircuitName       0
 avgLapTime_s      8
 stdLapTime_s     10
 GridPosition      0
 Position          0
 raceID            0
 Year              0
 dtype: int64,
 Driver            0
 lapsCompleted     0
 Team              0
 CircuitName       0
 avgLapTime_s     13
 stdLapTime_s     15
 GridPosition      0
 Position          0
 raceID            0
 Year              0
 dtype: int64,
 Driver            0
 lapsCompleted     0
 Team              0
 CircuitName       0
 avgLapTime_s     12
 stdLapTime_s     13
 GridPosition      0
 Position          0
 raceID            0
 Year              0
 dtype: int64)

In [7]:
for col in ['avgLapTime_s', 'stdLapTime_s']:
    stats_2023[col] = stats_2023[col].fillna(stats_2023[col].median())
    stats_2024[col] = stats_2024[col].fillna(stats_2024[col].median())
    stats_2025[col] = stats_2025[col].fillna(stats_2025[col].median())

In [8]:
stats_2023

,Driver,lapsCompleted,Team,CircuitName,avgLapTime_s,stdLapTime_s,GridPosition,Position,raceID,Year
0,ALB,57.0,Williams,Bahrain Grand Prix,100.465088,5.821969,15.0,10.0,1,2023
1,ALO,57.0,Aston Martin,Bahrain Grand Prix,99.567947,5.448373,5.0,3.0,1,2023
2,BOT,57.0,Alfa Romeo,Bahrain Grand Prix,100.164614,4.719029,12.0,8.0,1,2023
3,DEV,56.0,AlphaTauri,Bahrain Grand Prix,101.058196,5.017134,19.0,14.0,1,2023
4,GAS,57.0,Alpine,Bahrain Grand Prix,100.184018,6.135426,20.0,9.0,1,2023
...,...,...,...,...,...,...,...,...,...,...
433,SAR,58.0,Williams,Abu Dhabi Grand Prix,91.558879,4.195771,20.0,16.0,22,2023
434,STR,58.0,Aston Martin,Abu Dhabi Grand Prix,91.004414,3.689126,13.0,10.0,22,2023
435,TSU,58.0,AlphaTauri,Abu Dhabi Grand Prix,90.788138,2.808405,6.0,8.0,22,2023
436,VER,58.0,Red Bull Racing,Abu Dhabi Grand Prix,90.045241,3.834777,1.0,1.0,22,2023


- #### We now concat all the DataFrames:

In [9]:
# Use Pandas concat to make a big DataFrame with all stats:
all_stats = pd.concat([stats_2023, stats_2024, stats_2025], ignore_index = True)
# Sort the dataframe chronologically:
all_stats = all_stats.sort_values(by = ['Year', 'raceID']).reset_index(drop = True)

- #### We'll find the minimum average time per race, so we can normalize our values per driver

In [10]:
# Get the minimum average lap time per race
all_stats['perRaceMinAvgLapTime'] = all_stats.groupby(['Year', 'raceID'])['avgLapTime_s'].transform('min')
# Calculate the percentage difference for each entry, 
all_stats['avgLapTime_s_norm'] = (
    (all_stats['avgLapTime_s'] - all_stats['perRaceMinAvgLapTime']) / all_stats['perRaceMinAvgLapTime']
)
# Calculate the normalized standard deviation
all_stats['stdLapTime_s_norm'] = (
    all_stats['stdLapTime_s'] / all_stats['perRaceMinAvgLapTime']
)

In [11]:
# !!!!!!!!! CURRENT WINDOW SIZE !!!!!!!!!!
# !!!!!!!!! MAY NEED ADJUSTMENT !!!!!!!!!!
WINDOW_SIZE = 5

In [12]:
# Columns we want to calculate the historical mean for:
target_cols = ['avgLapTime_s_norm', 'stdLapTime_s_norm', 'GridPosition', 'Position']

for col in target_cols:

    # Group the data by driver:
    driver_stats = all_stats.groupby('Driver')[col]

    # Calculate the expanding mean up to the previous race:
    all_stats[f'Prev_Avg_{col}'] = driver_stats.transform(
        # Calculate the expanding mean. Shift one position to the left so that we do not include the current race
        # to the predictors table (X) and avoid data leakage. We use Pandas' .shift(1) for that.
        lambda x: x.expanding(min_periods = 1).mean().shift(1)
    )

    # Calculate the rolling mean (for a given window of races) up to the previous race:
    all_stats[f'Rolling_Prev_Avg_{col}'] = driver_stats.transform(
        # Calculate the rolling mean. Shift one position to the left so that we do not include the current race
        # to the predictors table (X) and avoid data leakage. We use Pandas' .shift(1) for that.
        lambda x: x.rolling(window = WINDOW_SIZE, min_periods = 1).mean().shift(1)
    )

# Drop potential new null values:
all_stats.dropna(
    subset = ['Prev_Avg_avgLapTime_s_norm', 'Prev_Avg_stdLapTime_s_norm', 'Prev_Avg_GridPosition', 'Prev_Avg_Position',
             'Rolling_Prev_Avg_avgLapTime_s_norm', 'Rolling_Prev_Avg_stdLapTime_s_norm',
              'Rolling_Prev_Avg_GridPosition', 'Rolling_Prev_Avg_Position'],
    inplace = True
)

- #### We'll do the same for the average (expanding and rolling) Final Position that a driver historically has on a circuit
  ##### We don't do it in our loop because we need to group them differently.

In [13]:
all_stats['Prev_Avg_Finish_Track'] = all_stats.groupby(['Driver', 'CircuitName'])['Position'].transform(
    lambda x: x.expanding(min_periods = 1).mean().shift(1)
)

all_stats['Rolling_Prev_Avg_Finish_Track'] = all_stats.groupby(['Driver', 'CircuitName'])['Position'].transform(
    lambda x: x.rolling(window = WINDOW_SIZE, min_periods = 1).mean().shift(1)
)

all_stats.dropna(
    subset = ['Prev_Avg_Finish_Track', 'Rolling_Prev_Avg_Finish_Track'],
    inplace = True
)

- #### Aggregate stats per Team:

In [14]:
# Find out how the team as a whole is doing in recent races when it comes to average lap times
all_stats['Rolling_Prev_Avg_TeamPace'] = all_stats.groupby('Team')['avgLapTime_s_norm'].transform(
    lambda x: x.rolling(window = WINDOW_SIZE, min_periods = 1).mean().shift(1)
)

# Create a "per_team" df for summed "Position"s
per_team_stats = all_stats.groupby(['Year', 'raceID', 'Team'])['Position'].mean().reset_index()
# Merge the new summed Positions (will rename them) with the original df
per_team_stats.rename(columns = {'Position': 'perRace_Team_Avg_Pos'}, inplace=True)
all_stats = all_stats.merge(
    per_team_stats[['Year', 'raceID', 'Team', 'perRace_Team_Avg_Pos']],
    on=['Year', 'raceID', 'Team'],
    how='left'
)

all_stats['Rolling_Prev_Avg_TeamFinalPos'] = all_stats.groupby('Team')['perRace_Team_Avg_Pos'].transform(
    lambda x: x.rolling(window = WINDOW_SIZE, min_periods = 1).mean().shift(1)
)

all_stats.dropna(
    subset = ['Rolling_Prev_Avg_TeamPace', 'Rolling_Prev_Avg_TeamFinalPos'],
    inplace = True
)

- #### Add a "bad result" column, to penalize drivers that consistently finish below 10th

In [15]:
all_stats['BadResult'] = (all_stats['Position'] > 10).astype(int)

In [16]:
all_stats.columns

Index(['Driver', 'lapsCompleted', 'Team', 'CircuitName', 'avgLapTime_s',
       'stdLapTime_s', 'GridPosition', 'Position', 'raceID', 'Year',
       'perRaceMinAvgLapTime', 'avgLapTime_s_norm', 'stdLapTime_s_norm',
       'Prev_Avg_avgLapTime_s_norm', 'Rolling_Prev_Avg_avgLapTime_s_norm',
       'Prev_Avg_stdLapTime_s_norm', 'Rolling_Prev_Avg_stdLapTime_s_norm',
       'Prev_Avg_GridPosition', 'Rolling_Prev_Avg_GridPosition',
       'Prev_Avg_Position', 'Rolling_Prev_Avg_Position',
       'Prev_Avg_Finish_Track', 'Rolling_Prev_Avg_Finish_Track',
       'Rolling_Prev_Avg_TeamPace', 'perRace_Team_Avg_Pos',
       'Rolling_Prev_Avg_TeamFinalPos', 'BadResult'],
      dtype='object')

- #### We can now construct our X dataset, uring the columns of interest:
  #### ('GridPosition', 'Prev_Avg_Lap_s', 'Prev_Std_Lap_s', 'Prev_Avg_Grid', 'Prev_Avg_Position')

In [17]:
X_cols = ['Prev_Avg_GridPosition', 'Prev_Avg_Position', 'Prev_Avg_avgLapTime_s_norm', 'Prev_Avg_stdLapTime_s_norm',
         'Rolling_Prev_Avg_avgLapTime_s_norm', 'Rolling_Prev_Avg_stdLapTime_s_norm',
         'Rolling_Prev_Avg_GridPosition', 'Rolling_Prev_Avg_Position', 'Prev_Avg_Finish_Track', 'Rolling_Prev_Avg_Finish_Track',
         'Rolling_Prev_Avg_TeamPace', 'Rolling_Prev_Avg_TeamFinalPos', 'BadResult']
X = all_stats[X_cols].copy()
X.head()

,Prev_Avg_GridPosition,Prev_Avg_Position,Prev_Avg_avgLapTime_s_norm,Prev_Avg_stdLapTime_s_norm,Rolling_Prev_Avg_avgLapTime_s_norm,Rolling_Prev_Avg_stdLapTime_s_norm,Rolling_Prev_Avg_GridPosition,Rolling_Prev_Avg_Position,Prev_Avg_Finish_Track,Rolling_Prev_Avg_Finish_Track,Rolling_Prev_Avg_TeamPace,Rolling_Prev_Avg_TeamFinalPos,BadResult
7,14.217391,15.434783,0.046673,0.084685,0.054913,0.063582,13.6,16.4,10.0,10.0,0.023036,11.0,1
9,11.608696,12.086957,0.045480,0.083827,0.018699,0.077631,15.2,10.6,8.0,8.0,0.297461,16.5,1
11,9.565217,10.782609,0.025609,0.093353,0.012992,0.081483,9.2,9.2,15.0,15.0,0.013351,6.0,0
12,6.304348,7.956522,0.031651,0.087879,0.016761,0.089632,5.2,7.4,4.0,4.0,0.014388,7.5,0
13,16.869565,16.000000,0.049493,0.093524,0.023692,0.087703,16.4,15.8,16.0,16.0,0.035938,12.5,1


- #### Create some ID columns that will be used for identification purposes

In [18]:
ID_cols = ['Driver', 'Year', 'raceID']
ID = all_stats[ID_cols].copy() 

- #### And finally, our target variable

In [19]:
all_stats['Winner'] = (all_stats['Position'] == 1).astype(int) 
y = all_stats['Winner'].copy()

- #### We shouldn't shuffle our data, so we can choose as X_train the whole 2023 and 2023 seasons:

In [20]:
train_idx = np.where(ID['Year'] < 2025)[0]
test_idx = np.where(ID['Year'] == 2025)[0]
X_train, X_test = X.values[train_idx], X.values[test_idx]
y_train, y_test = y.values[train_idx], y.values[test_idx]

- #### Scale our data using Sklearn's StandardScaler:

In [21]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

- #### Create a simple logistic regression model:

In [22]:
lr = LogisticRegression(class_weight = 'balanced', random_state = 42)
lr.fit(X_train_sc, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


- #### We used class_weight as 'balanced' because our classes are not at all evenly distributed. As a matter of fact, our positive class is very rare.

In [23]:
y.value_counts()

Winner
0    591
1     39
Name: count, dtype: int64

In [24]:
lr.score(X_test_sc, y_test)

0.83203125

In [25]:
lr.score(X_train_sc, y_train)

0.8181818181818182

- #### Our accuracy is a bit too high. This could be because the Negative ("0") class is too easy to predict. If you predict every single driver as a loser, you'll still have 1.184/1.247 = 0,901363 accuracy!!

In [26]:
y_pred = lr.predict(X_test_sc)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.84      0.90       238
           1       0.24      0.67      0.36        18

    accuracy                           0.83       256
   macro avg       0.61      0.76      0.63       256
weighted avg       0.92      0.83      0.87       256



- #### Move on to a __Random Forest__ model

Random Forests are sensitive to several parameters. That's why we'll do a Grid Search first.

In [27]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, 20, 50, 80, 150, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 5],
    'class_weight': ['balanced']
}

grid = GridSearchCV(RandomForestClassifier(), param_grid = param_grid, scoring = 'roc_auc', cv = 5)
grid.fit(X_train, y_train)
grid.best_params_

{'class_weight': 'balanced',
 'max_depth': None,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 50}

In [28]:
rf = grid.best_estimator_
rf.fit(X_train, y_train)

,n_estimators,50
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [29]:
rf.score(X_test, y_test)

0.90234375

In [30]:
rf.score(X_train, y_train)

0.9598930481283422

In [31]:
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       238
           1       0.35      0.44      0.39        18

    accuracy                           0.90       256
   macro avg       0.65      0.69      0.67       256
weighted avg       0.91      0.90      0.91       256



In [32]:
neg_count = sum(y_train == 0)
pos_count = sum(y_train == 1)
scale_weight = neg_count / pos_count

xgbc = xgb.XGBClassifier(
    objective = 'binary:logistic',
    random_state = 42,
    eval_metric = 'logloss',
    scale_pos_weight = scale_weight
)

xgbc.fit(X_train_sc, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [33]:
y_pred_xgb = xgbc.predict(X_test_sc)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       238
           1       0.38      0.33      0.35        18

    accuracy                           0.91       256
   macro avg       0.66      0.65      0.65       256
weighted avg       0.91      0.91      0.91       256



- #### There's a possible overfitting. Let's try and tune our XGB Model

In [34]:
param_dist = {
    'learning_rate': stats.uniform(loc = 0.01, scale = 0.1),
    'n_estimators': stats.randint(100, 1000),
    'scale_pos_weight': [scale_weight, scale_weight * 1.1, scale_weight * 0.9],
    'max_depth': stats.randint(1, 20),
    'min_child_weight': stats.randint(3, 7),
    'gamma': stats.uniform(loc = 0, scale = 0.5)
}

random_search = RandomizedSearchCV(
    estimator = xgb.XGBClassifier(objective='binary:logistic', eval_metric = 'logloss', seed = 42),
    n_iter = 100, param_distributions = param_dist, cv = 5, scoring = 'roc_auc', verbose = 1, random_state = 42)

random_search.fit(X_train, y_train)
random_search.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


{'gamma': np.float64(0.16039003248586792),
 'learning_rate': np.float64(0.028651851039985424),
 'max_depth': 3,
 'min_child_weight': 6,
 'n_estimators': 863,
 'scale_pos_weight': np.float64(15.12857142857143)}

In [35]:
better_xgbc = random_search.best_estimator_
better_xgbc.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [36]:
y_pred_better_xgb = better_xgbc.predict(X_test)
print(classification_report(y_test, y_pred_better_xgb))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96       238
           1       0.45      0.50      0.47        18

    accuracy                           0.92       256
   macro avg       0.71      0.73      0.72       256
weighted avg       0.93      0.92      0.92       256

